In [ ]:
from scipy.integrate import odeint
import numpy as np
import matplotlib.pyplot as plt
import SA
import main
import HC

In [ ]:
def lotka_volterra(y, t, alpha, beta, delta, gamma):
    x, y = y
    dxdt = alpha * x - beta * x * y
    dydt = delta * x * y - gamma * y
    return [dxdt, dydt]

def load_data():
    data = np.loadtxt("../observed_data/predator-prey-data.csv", delimiter=",")
    return data[:, 0], data[:, 1], data[:, 2]

def plot_model(t_data, x_data, y_data, sim_data):
    """
    Plot the observed data and the best fit
    """
    prey, predator = sim_data
    
    fig, axis = plt.subplots(1, 2, figsize=(12, 6))
    axis[0].plot(t_data, x_data, 'o')
    axis[0].plot(t_data, y_data, 'o')
    axis[0].set_title("Observed Data")

    axis[1].plot(t_data, prey, label='Prey')
    axis[1].plot(t_data, predator, label='Predator')
    axis[1].set_title("Lotka-Volterra Model")

    fig.suptitle(f"Lotka-Volterra Model\n given by $\\alpha = 0.57104693$, $\\beta = 0.47572982$, $\\delta = 0.93209038$, $\\gamma = 1.1220162$")
    fig.supxlabel("Time")
    fig.supylabel("Population")
    fig.legend()

    plt.tight_layout()
    plt.gcf().set_dpi(300)
    # plt.savefig("../visualization/lotka_volterra.png")
    plt.show()

def plot_mse(N_boot, max_boot, objective, initial_params, proposal_func, max_iter, tol):
    boot_range = np.array(range(0, max_boot))
    mse = np.zeros(max_boot)
    
    for i in boot_range:
        mean_mse = 0
        for _ in range(N_boot):
            boot_data = bootstrapping(i)
            _, best_mse = HC.hill_climbing(
            objective, proposal_func, boot_data, initial_params, max_iter, tol
            )
            mean_mse += best_mse
        mse[i] = mean_mse / N_boot
    
    plt.plot(boot_range, mse)
    plt.xlabel("Number of Data Points Removed")
    plt.ylabel(f"The Mean Mean Squared Error over {N_boot} Bootstraps")
    plt.title("Mean Squared Error vs. Number of Data Points")
    plt.savefig("../visualization/mse_vs_data_points.png")
    plt.show()

def bootstrapping(num_data_points):
    """
    Perform bootstrapping to generate new data
    """
    true_data = load_data()
    length = len(true_data[0])

    indices = np.random.choice(length, num_data_points, replace=False)
    result_data = tuple(
        np.delete(data, indices) for data in true_data
    )
    return result_data

In [ ]:
t_data, x_data, y_data = load_data()

y0 = [x_data[0], y_data[0]]
alpha, beta, delta, gamma = [0.57104693,0.47572982,0.93209038,1.1220162]

solution = odeint(lotka_volterra, y0, t_data, args=(alpha, beta, delta, gamma))
prey = solution[:, 0]
predator = solution[:, 1]

plot_model(t_data, x_data, y_data, [prey, predator])

In [ ]:
max_boot = 100
objective = main.objective
initial_params = np.array([1.0, 0.1, 0.1, 1.0])
proposal_func = main.proposal_func
max_iter = 100
tol = 1e-6

plot_mse(40, max_boot, objective, initial_params, proposal_func, max_iter, tol);